In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [2]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import gdata_dir, scratch_dir
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.topography import pysheds_accumulation, catchment_gullies, catchment_ridges, calculate_slope

Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_Ssoil_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_Ssoil_2021.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_GPP_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_GPP_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_ozwald_8day.nc


In [3]:
# Choosing location
lat, lon = -34.3890427, 148.469499
buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total
stub = "MILG_1km"
start_year = 2020
end_year = 2021

# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = os.path.join(scratch_dir, "tmp")

In [4]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_terrain_original.tif
Saved /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_terrain_cleaned.tif
CPU times: user 102 ms, sys: 14.1 ms, total: 116 ms
Wall time: 291 ms


In [5]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable unfortunately
# variables = ['Clay', 'Silt', 'Sand', 'pH_CaCl2', 'Bulk_Density', 'Available_Water_Capacity', 'Effective_Cation_Exchange_Capacity', 'Total_Nitrogen', 'Total_Phosphorus']
variables = ['Clay', 'Sand']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Downloaded /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Clay.tif


XMLSyntaxError: AttValue: " or ' expected, line 1, column 27 (<string>, line 1)

In [6]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Maximum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Minimum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Annual_Rainfall_2020_2021_average.tif
CPU times: user 153 ms, sys: 21.5 ms, total: 175 ms
Wall time: 940 ms


In [7]:
%%time
# Download modelled variables from OzWald
variables = ['SN', 'Ssoil']
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_SN_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_SN_2021.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_Ssoil_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_Ssoil_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_ozwald_8day.nc
CPU times: user 114 ms, sys: 10.2 ms, total: 124 ms
Wall time: 2.49 s


In [8]:
%%time
# Download daily weather data hosted on OzWald
variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, 0.1, start_year, end_year, outdir, "MILG_20km", tmp_dir)

Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_VPeff_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_VPeff_2021.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_Uavg_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_Uavg_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_20km_ozwald_daily.nc
CPU times: user 93.9 ms, sys: 6.59 ms, total: 101 ms
Wall time: 1.28 s
